# Processamento de Vídeo utilizando o OpenCV

In [1]:
import cv2             # OpenCV
import numpy as np

### 1. Leitura e processamento de um arquivo de vídeo 

In [4]:
def green_boost(img, gamma):
    '''Enaltece o canal verde de uma imagem através de uma transformação 
    pontual de lei de potência.'''
    
    green_channel = img[:,:,1]
    new_green_channel = np.power(green_channel, gamma)
    new_green_channel_normalized = 255*new_green_channel/np.max(new_green_channel)
    new_green_channel_normalized = np.round(new_green_channel_normalized).astype(np.uint8)
    new_img = img.copy()
    new_img[:,:,1] = new_green_channel_normalized
    
    return new_img

# Abre arquivo de vídeo
vcap = cv2.VideoCapture('video.mp4')

while True:

    # Obtém próximo frame do vídeo. ret indica se o frame
    # foi lido com sucesso, curr_frame possui a imagem
    ret, curr_frame = vcap.read()
    if ret==True:
        # Aqui podemos aplicar métodos de processamento de imagem na imagem curr_frame. 
        # Por exemplo, enaltecer o canal verde
        new_frame = green_boost(curr_frame, gamma=0.5)
        cv2.imshow('frame',new_frame)
        
    # cv2.waitKey(x) faz o programa esperar x milisegundos para que uma tecla 
    # seja digitada. 
    # - Dependendo da plataforma, o resultado dessa função pode ser um número com mais 
    # de 8 bits. É comum que uma operação bitwise and seja feita entre o resultado 
    # dessa função e o número 0xff (11111111), de forma que apenas os 8 primeiros bits 
    # sejam mantidos. 
    # - Em alguns casos, o vídeo será mostrado em uma velocidade muito grande. Se isso
    # ocorrer, é recomendável utilizar cv2.waitKey(33), que significa esperar 33 milisegundos 
    # para que uma tecla seja digitada. Isso equivale a mostrar 30 imagens por segundo (1/0.033).
    # Portanto, o FPS (frames per second) do vídeo mostrado será aproximadamente 30. O 
    # valor não é exatamente 30 porque um tempo é gasto no processamento do frame
    key = cv2.waitKey(1) & 0xff
    
    # A tecla esc corresponde ao valor 27
    if key == 27:
        break

# Libera o arquivo
vcap.release()
# Fecha a janela
cv2.destroyAllWindows()

### 2. Leitura e processamento utilizando uma câmera

In [7]:
import scipy.ndimage as ndi

def process_frame(img):
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = img_gray.astype(float)
    grad_mag = ndi.gaussian_gradient_magnitude(img_gray, sigma=2)
    grad_mag_norm = grad_mag - np.min(grad_mag)
    grad_mag_norm = 255*grad_mag_norm/np.max(grad_mag_norm)
    grad_mag_norm = np.round(grad_mag_norm).astype(np.uint8)    
    return grad_mag_norm
    
# Abre dispositivo de captura. O parâmetro 0 faz com que
# a primeira câmera encontrada seja utilizada
vcap = cv2.VideoCapture(1)

while True:
    ret, curr_frame = vcap.read()
    if ret==True:
        # Podemos redimensionar a imagem para diminuir o custo 
        # computacional necessário para processar os frames
        #resized_frame = cv2.resize(curr_frame, (640, 360))
        processed_frame = process_frame(curr_frame)
               
        cv2.imshow('frame', curr_frame)
    else:
        print("Frame not available")
        
    key = cv2.waitKey(1) & 0xff

    if key == 27:
        break

vcap.release()
cv2.destroyAllWindows()